In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
from statsmodels.formula.api import ols

DATA UNDERSTANDING AND PREPROCESSING

Operations:

    1. Checked data types, column names, and unique values.
    
    2. Verified there were no missing values in the dataset.

Basic Data Exploration

In [ ]:
df = pd.read_csv(r"datasets\raw.csv")

# Additional data exploration beyond what was shown
# Check data types and verify no unexpected values
print(df['Neighborhood_Quality'].unique())
print(df.columns)

In [ ]:
# Create derived features that might be helpful
df['Age'] = 2025 - df['Year_Built']  # Current year minus year built
df['Price_Per_SqFt'] = df['House_Price'] / df['Square_Footage']

In [ ]:
df.head(5)

Data Cleaning Verification

In [ ]:
def outliers(df, column):
    Q1 = np.percentile(df[column], 25) 
    Q3 = np.percentile(df[column], 75) 

    IQR=Q3-Q1

    LF= Q1-1.5*(IQR)
    UF= Q3+1.5*(IQR)

    outlier=df[(df[column]>UF) & (df[column]<LF)]
    return outlier 

for i in df.columns:
    if(df[i].dtypes in ("int64","float64")):
        print(f"Number of {i} outliers: {len(outliers(df,i))}")

Conclusions:

    1.The dataset is clean, with no missing values.

    2.The Neighborhood_Quality column is categorical, while other columns are numerical.

    3.Derived features like Age (based on Year_Built) and Price_Per_SqFt (based on Square_Footage and House_Price) can provide additional insights.

UNIVARIATE ANALYSIS

Operations:

    1.Analyzed statistical summaries (mean, median, standard deviation, skewness, kurtosis).

    2.Plotted histograms for numerical features.



Numerical Features Distribution

In [ ]:
# Statistical summary with additional metrics

numerical_features=['Square_Footage', 'Num_Bedrooms', 'Num_Bathrooms','Year_Built', 'Lot_Size', 'Garage_Size', 'House_Price']

for feature in numerical_features:
    print(f"\nSummary for {feature}:")
    summary = pd.DataFrame({
        'Mean': [np.mean(df[feature])],
        'Median': [np.median(df[feature])],
        'Std Dev': [np.std(df[feature])],
        'Skewness': [df[feature].skew()],
        'Kurtosis': [df[feature].kurt()],
        'Min': [np.min(df[feature])],
        'Max': [np.max(df[feature])]
    })
    print(summary)

In [ ]:
# Set up visualization style

sns.set_style('whitegrid')
sns.set_palette("Set2")

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(20, 30))
axes = axes.flatten()

for i, feature in enumerate(numerical_features):
    if i < len(axes):
        sns.histplot(df[feature], kde=True, ax=axes[i])
        axes[i].set_title(f'Distribution of {feature}')
        axes[i].set_xlabel(feature)
        
plt.tight_layout()
plt.show()

Conclusions:

    1.Square Footage: Right-skewed distribution; most houses are smaller in size, with a few very large properties.

    2.Number of Bedrooms/Bathrooms: Most houses have 2–4 bedrooms and 1–3 bathrooms.

    3.Year Built: Majority of houses were built after 1950, with a peak in recent decades.

    4.Lot Size: Highly skewed; most houses have small lots, but a few have very large ones.

    5.House Price: Right-skewed; most houses fall within a moderate price range, with some high-value outliers.

    6.Skewness in key features suggests potential need for transformations during modeling.

CATEGORICAL FEATURE ANALYSIS

Operations:

    1.Analyzed the distribution of Neighborhood_Quality.

    2.Examined average house prices for each neighborhood quality level using bar plots and box plots.

In [ ]:
# Analyze neighborhood quality distribution
neighborhood_counts = df['Neighborhood_Quality'].value_counts()
plt.figure(figsize=(10, 6))
sns.barplot(x=neighborhood_counts.index, y=neighborhood_counts.values)
plt.title('Distribution of Houses by Neighborhood Quality')
plt.ylabel('Number of Houses')
plt.xlabel('Neighborhood Quality')
plt.show()

In [ ]:
# Average house price by neighborhood
neighborhood_price = df.groupby('Neighborhood_Quality')['House_Price'].agg(['mean', 'median', 'std', 'count'])
print(neighborhood_price)

In [ ]:
# Visualize price distribution by neighborhood
plt.figure(figsize=(10, 6))
sns.boxplot(x='Neighborhood_Quality', y='House_Price', data=df)
plt.title('House Price Distribution by Neighborhood Quality')
plt.ylabel('House Price ($)')
plt.xlabel('Neighborhood Quality')
plt.show()

Conclusions:

    1.Neighborhood quality is well-distributed across categories.

    2.Higher neighborhood quality correlates with higher average house prices.

    3.Significant price variability exists within each neighborhood quality level, indicating other factors also influence prices.

 BIVARIATE ANALYSIS

Operations:

    1.Correlation heatmap to assess relationships between numerical features.

    2.Scatter plots and regression lines for key variables against house price.

Correlation Analysis

In [ ]:
# Correlation heatmap
numeric_df = df.select_dtypes(include=[np.number])
plt.figure(figsize=(12, 10))
correlation_matrix = numeric_df.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, annot=True, mask=mask, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap of Numerical Features')
plt.show()

In [ ]:
# Detailed price correlation analysis
price_correlations = correlation_matrix['House_Price'].sort_values(ascending=False)
print("Correlations with House Price:")
print(price_correlations)

In [ ]:
# Key scatter plots with regression lines
key_features = ['Square_Footage', 'Lot_Size', 'Age']
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, feature in enumerate(key_features):
    sns.regplot(x=feature, y='House_Price', data=df, ax=axes[i])
    axes[i].set_title(f'{feature} vs House Price')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('House Price ($)')
    
plt.tight_layout()
plt.show()

In [ ]:
# Categorical feature relationships
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.boxplot(x='Num_Bedrooms', y='House_Price', data=df, ax=axes[0])
axes[0].set_title('Price by Number of Bedrooms')
axes[0].set_xlabel('Number of Bedrooms')
axes[0].set_ylabel('House Price ($)')

sns.boxplot(x='Num_Bathrooms', y='House_Price', data=df, ax=axes[1])
axes[1].set_title('Price by Number of Bathrooms')
axes[1].set_xlabel('Number of Bathrooms')
axes[1].set_ylabel('House Price ($)')

sns.boxplot(x='Garage_Size', y='House_Price', data=df, ax=axes[2])
axes[2].set_title('Price by Garage Size')
axes[2].set_xlabel('Garage Size')
axes[2].set_ylabel('House Price ($)')

plt.tight_layout()
plt.show()


Conclusions:

    1.Square Footage has the strongest positive correlation with house price (r≈0.8).

    2.Lot Size, Number of Bathrooms, and Garage Size also show moderate positive correlations with house price.

    3.Weak or no correlation between Year Built and house price suggests that age alone is not a strong predictor of value.

    4.Outliers in features like Lot Size and House Price may influence correlations and require careful handling during modeling.

MUTLIVARIATE ANALYSIS

Operations:

    1.Pair plots to visualize relationships among multiple variables.

    2.Joint plots to analyze interactions between square footage, house price, and neighborhood quality.

Joint Plots and Pair Plots

In [ ]:
# Create a pair plot for key variables
key_vars = ['Square_Footage', 'Num_Bedrooms', 'Num_Bathrooms', 'Lot_Size', 'House_Price']
sns.pairplot(df[key_vars], height=2.5)
plt.suptitle('Pairwise Relationships Between Key Variables', y=1.02)
plt.show()

In [ ]:

# Advanced joint plot with neighborhood quality
g = sns.JointGrid(data=df,x="Square_Footage",y="House_Price",hue="Neighborhood_Quality",height=8,palette="viridis"   )
g.plot_joint(sns.scatterplot, hue="Neighborhood_Quality", alpha=0.7)
g.plot_marginals(sns.kdeplot, fill=True)
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Joint Distribution of Square Footage and House Price by Neighborhood Quality')
plt.show()

3D Visualizations

In [ ]:

fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

xs = df['Square_Footage']
ys = df['Lot_Size']
zs = df['House_Price']
ax.scatter(xs, ys, zs, s=50, alpha=0.6, c=df['House_Price'], cmap='viridis')

ax.set_xlabel('Square Footage')
ax.set_ylabel('Lot Size')
ax.set_zlabel('House Price')
ax.set_title('3D Visualization of Square Footage, Lot Size, and House Price')
plt.show()

Conclusions:

    1.Larger homes (higher square footage) in high-quality neighborhoods command significantly higher prices.

    2.Interaction effects between square footage and neighborhood quality suggest that location amplifies the impact of home size on price.

    3.Some features (e.g., lot size) show non-linear relationships with house price, indicating potential for advanced modeling techniques like polynomial regression.

ADVANCED EDA TECHNIQUES

Time-Based Analysis

Operations:

    1.Grouped houses by decade built to analyze trends in pricing and construction patterns over time.

In [ ]:
# Group houses by decade built
df['Decade_Built'] = (df['Year_Built'] // 10) * 10
decade_stats = df.groupby('Decade_Built').agg({
    'House_Price': ['mean', 'median', 'std', 'count'],
    'Square_Footage': 'mean'
})
print(decade_stats)


In [ ]:

# Visualize price trends over time
plt.figure(figsize=(12, 6))
sns.lineplot(x='Year_Built', y='House_Price', data=df, ci=None, estimator='mean')
plt.title('Average House Price by Year Built')
plt.xlabel('Year Built')
plt.ylabel('Average House Price ($)')
plt.grid(True)
plt.show()

Conclusions:

    1.Houses built in more recent decades tend to have higher average prices due to modern amenities or larger sizes.

    2.Construction peaked in certain decades (e.g., 1980s or 2000s), reflecting housing market trends during those periods.

    3.Older houses may still command high prices if located in desirable neighborhoods or renovated extensively.

Price Per Square Foot Analysis

Operations:

    1.Analyzed the distribution of price per square foot across neighborhoods using box plots and histograms.

In [ ]:
# Analyze price per square foot
plt.figure(figsize=(12, 6))
sns.boxplot(x='Neighborhood_Quality', y='Price_Per_SqFt', data=df)
plt.title('Price Per Square Foot by Neighborhood Quality')
plt.xlabel('Neighborhood Quality')
plt.ylabel('Price Per Square Foot ($)')
plt.grid(True)
plt.show()

In [ ]:
# Histogram of price per square foot
plt.figure(figsize=(10, 6))
sns.histplot(df['Price_Per_SqFt'], kde=True)
plt.title('Distribution of Price Per Square Foot')
plt.xlabel('Price Per Square Foot ($)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

Conclusions:

    1.Higher-quality neighborhoods generally have higher price-per-square-foot values.

    2.Significant overlap exists between mid-tier neighborhoods, suggesting other factors like lot size or age influence pricing beyond location alone.

    3.Outliers in price-per-square-foot values could represent luxury properties or data anomalies.

Feature Interaction Effects

In [ ]:
# Create interaction plots
plt.figure(figsize=(14, 8))
for quality in df['Neighborhood_Quality'].unique():
    subset = df[df['Neighborhood_Quality'] == quality]
    plt.scatter(subset['Square_Footage'], subset['House_Price'], 
                label=quality, alpha=0.7)
    
    # Add trendline for each quality
    z = np.polyfit(subset['Square_Footage'], subset['House_Price'], 1)
    p = np.poly1d(z)
    plt.plot(subset['Square_Footage'], p(subset['Square_Footage']), 
             linestyle='--', linewidth=2)

plt.title('Square Footage vs House Price by Neighborhood Quality')
plt.xlabel('Square Footage')
plt.ylabel('House Price ($)')
plt.legend()
plt.grid(True)
plt.show()


STATISTICAL ANALYSIS AND FEATURE IMPORTANCE

ANOVA Test for Categorical Variables

Operations:

    1.Conducted an ANOVA test to assess whether neighborhood quality significantly impacts house prices.

In [ ]:
# ANOVA test for neighborhood quality
groups = [df[df['Neighborhood_Quality'] == quality]['House_Price'] 
          for quality in df['Neighborhood_Quality'].unique()]
f_stat, p_value = stats.f_oneway(*groups)
print(f"ANOVA Test for Neighborhood Quality: F-statistic = {f_stat}, p-value = {p_value}")

In [ ]:
# Multiple comparison test if ANOVA is significant
if p_value < 0.05:
    tukey = pairwise_tukeyhsd(df['House_Price'], df['Neighborhood_Quality'], alpha=0.05)
    print(tukey)

Conclusions:

    1.The ANOVA test revealed a statistically significant difference in house prices across different neighborhood quality levels (p<0.05).

    2.Post-hoc tests (e.g., Tukey's HSD) showed that high-quality neighborhoods differ significantly from low-quality ones in terms of pricing.

LINEAR REGRESSION ANALYSIS

Operations:

    1.Built a linear regression model to identify the most important predictors of house price.

In [ ]:
# Encode categorical variables (e.g., Neighborhood_Quality)
df_encoded = pd.get_dummies(df, columns=['Neighborhood_Quality'], drop_first=True)

# Convert all features to float to avoid dtype issues
X = df_encoded.drop('House_Price', axis=1).astype(float)
X = sm.add_constant(X)  # Add intercept term
y = df_encoded['House_Price'].astype(float)

In [ ]:
print(X.dtypes)
print(y.dtypes)

In [ ]:
# Check for missing values
print(df_encoded.isnull().sum())

# Drop or impute missing values
df_encoded = df_encoded.dropna()  # Example: drop rows with missing values


In [ ]:
model = sm.OLS(y.astype(float), X.astype(float)).fit()
print(model.summary())

In [ ]:
# Create feature importance visualization
coefficients = pd.DataFrame(model.params.values, index=model.params.index, columns=['Coefficient'])
coefficients = coefficients.iloc[1:]  # Remove intercept
coefficients['Absolute'] = abs(coefficients['Coefficient'])
coefficients = coefficients.sort_values('Absolute', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x=coefficients['Coefficient'], y=coefficients.index)
plt.title('Feature Coefficients in Linear Regression Model')
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.axvline(x=0, color='black', linestyle='--')
plt.grid(True)
plt.show()

Conclusions:

    1.Square footage emerged as the most important predictor of house price, followed by neighborhood quality and lot size.

    2.Number of bedrooms had a weaker impact compared to bathrooms or garage size, likely due to diminishing returns on additional bedrooms beyond a certain point.

    3.Multicollinearity among features like square footage and lot size needs careful handling during modeling.

Summary of Key Insights

    1.Square footage is the strongest driver of house prices, followed by neighborhood quality and lot size.

    2.Neighborhood quality significantly impacts pricing but interacts with other factors like square footage and lot size.

    3.Features like year built or number of bedrooms have less predictive power compared to other variables.

    4.Skewness in numerical features suggests potential transformations for better model performance.

    5.Outliers exist in several features (e.g., lot size, house price) and should be addressed during preprocessing.